In [5]:
# Imports
import os
import json
import time
import tiktoken
from openai import AzureOpenAI
from dotenv import load_dotenv

# Environment setup
load_dotenv()

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
key = os.getenv("AZURE_SEARCH_KEY") 
verbose = False #Set to true to see more output information

#Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  
  api_version = "2023-12-01-preview"
  )

messages=[]

def count_tokens(prompt) -> int:  
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    token_sizes = len(encoding.encode(prompt))
    return token_sizes

In [6]:
# system message
system_message = f'''
You are an assistant in charge of gathering the correctness and completion of provide informations from the user and, in case they are not provide a set of question to be asked to the user.
If all the requisites are provided, please reply with the action to be executed including the name of the libraries to be used.
To select the libraries to be used, please use the following list:
  - To open or close something: use the library "SE_OPEN_CLOSE"
  - To activate or deactivate something: use the library "SE_ACTIVATE_DEACTIVATE"
  - To load or trigger something: use the library "SE_LOAD_TRIGGER"
  - To cut something: use the library "SE_CUT"
  - To print something: use the library "SE_PRINT_262" for machine "M262" and "SE_PRINT_262" for machine "M263"


These are the input requirements to be verified:

1. The user must provide one or more actions to be executed (e.g. opening, closing, activating, load, trigger and so on).
2. If no action indicated please reply "What action do you want to perform?"
3. The user must provide one or more objects that the action will operate on (e.g. material, light, fan, and so on).
4. The user has to indicate the name of the PLC to use, if not provided please:
  - Reply with "What is the name of the PLC to use?"
  - Do not ask any other question until an the PLC name is provided.
4. If action is cut the following info have to be provided:
  - The lenght of the cut
  - How wide is the cut
  - The depth of the cut
  - The material to be cut
5. If action is to print someting the following info have to be provided: 
  - What has to be printed
  - Where the content has to be printed
  - The material to be printed
  - The color of the print
  - The size of the print
  - The number of copies


In the reply don't include any other text/info other than question regarding the missing prerequisites to fullfill the requisites.
'''

In [7]:
user_command = f'''
Hi, how are you doing today?

'''

Theses are the user commands, edit and run them to see the different output considering that output also depends on code implemented into function (to be replaced by RAG)

In [8]:
messages.append({'role': 'system', 'content': system_message})
llm_response=""

while True:
        
        user_command = input()
        if user_command == "exit":
            break
        if user_command != "":
            print(f"-> User: {user_command}", flush=True)
            messages.append({'role': 'user', 'content': user_command})   
        
        openai_response = client.chat.completions.create(
                model=deployment,    
                messages = messages,
                temperature=0.7,
                max_tokens=800,
                top_p=0.95,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None)
        result= openai_response.choices[0].message
        llm_response = result.content  
        print(f"---->LLM: {llm_response}", flush=True)
        messages.append({'role': 'assistant', 'content': result.content})     
        time.sleep(0.5)

-> User: Hello
---->LLM: What action do you want to perform?
-> User: i want to print
---->LLM: What has to be printed? Where should the content be printed? What is the material to be printed on? What is the color of the print? What is the size of the print? How many copies do you need? What is the name of the PLC to use?
-> User: 
---->LLM: What is the name of the PLC to use?
